# Answer prediction model E.ON + create pickle file 

Just run this notebook to create the pickle file.

In [36]:
import pandas as pd
import numpy as np
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = stopwords.words('german')
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\isele\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


### Processing data

In [32]:
# Importing data set saved locally

path_local=r"C:\Users\isele\OneDrive\Dokumente\GitHub\Graduation_Project_Matthias_Isele\notebooks\df_ml.csv"

path=path_local
df_ml=pd.read_csv(path, index_col=0)

df_ml_EON=df_ml[df_ml['Company']== 'E.ON Energie Deutschland GmbH']

# Categorizing answers of EON Energy
answer_list=df_ml_EON['Answer'].unique()
df_answer_category = pd.DataFrame(answer_list, columns=['Answer'])
df_answer_category['Answer_label']=[1,0,1,0,1,0]
df_answer_category=df_answer_category.set_index('Answer')

# Creating boolean answer label column
df_ml_EON['Answer_label']=df_ml_EON['Answer'].apply(lambda x: df_answer_category['Answer_label'][x])

# Replacing all non letters with empty space in Comment and Headline
# Transfer letters to lowercase and remove words of length 2 or less

df_ml_EON['Comment_alpha']=df_ml_EON['Comment']\
  .apply(lambda x: re.sub(r'[^a-zA-ZäüöÄÜÖß]+', ' ', x).lower())\
  .apply(lambda x: re.sub(r'\s[a-zäöü]{1}\s', ' ', x))\
  .apply(lambda x: re.sub(r'\s[a-zäöü]{2}\s', ' ', x))

df_ml_EON['Headline_alpha']=df_ml_EON['Headline']\
  .apply(lambda x: re.sub(r'[^a-zA-ZäüöÄÜÖß]+', ' ', x).lower())\
  .apply(lambda x: re.sub(r'\s[a-zäöü]{1}\s', ' ', x))\
  .apply(lambda x: re.sub(r'\s[a-zäöü]{2}\s', ' ', x))


# Remove stop words of Comment and Headline
# translate special characters

special_char_map = {ord('ä'):'ae', ord('ü'):'ue', ord('ö'):'oe', ord('ß'):'ss'}


df_ml_EON['Comment_no_stopwords']=df_ml_EON['Comment_alpha']\
  .apply(lambda x: ' '.join([word for word in x.split() if word not in (stop_words)]).translate(special_char_map))
df_ml_EON['Headline_no_stopwords']=df_ml_EON['Headline_alpha']\
  .apply(lambda x: ' '.join([word for word in x.split() if word not in (stop_words)]).translate(special_char_map))

#df_ml_EON[['Comment_no_stopwords','Comment_alpha','Comment','Headline_no_stopwords','Headline_alpha','Headline']]
df_ml_EON.info()

C:\Users\isele\AppData\Local\Temp\ipykernel_15900\317903312.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_ml_EON['Answer_label']=df_ml_EON['Answer'].apply(lambda x: df_answer_category['Answer_label'][x])
C:\Users\isele\AppData\Local\Temp\ipykernel_15900\317903312.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_ml_EON['Comment_alpha']=df_ml_EON['Comment']\
C:\Users\isele\AppData\Local\Temp\ipykernel_15900\317903312.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a s

<class 'pandas.core.frame.DataFrame'>
Index: 4965 entries, 5 to 1
Data columns (total 18 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Headline               4965 non-null   object 
 1   Comment                4965 non-null   object 
 2   Answer                 4965 non-null   object 
 3   Company                4965 non-null   object 
 4   Words_Comment          4965 non-null   float64
 5   Words_Headline         4965 non-null   int64  
 6   Words_Answer           4965 non-null   float64
 7   log_Words_Headline     4965 non-null   float64
 8   log_Words_Comment      4965 non-null   float64
 9   log_Words_Answer       4965 non-null   float64
 10  Stars_min_max_scaled   4965 non-null   float64
 11  Stars_geq3_TF          4965 non-null   int64  
 12  Stars_geq4_TF          4965 non-null   int64  
 13  Answer_label           4965 non-null   int64  
 14  Comment_alpha          4965 non-null   object 
 15  Headline_alp

C:\Users\isele\AppData\Local\Temp\ipykernel_15900\317903312.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_ml_EON['Comment_no_stopwords']=df_ml_EON['Comment_alpha']\
C:\Users\isele\AppData\Local\Temp\ipykernel_15900\317903312.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_ml_EON['Headline_no_stopwords']=df_ml_EON['Headline_alpha']\


### Train model

In [40]:
%%time
#Choose data and target
#data_columns=['Comment_no_stopwords','Headline_no_stopwords']
data_columns=['Comment_no_stopwords']
data=df_ml_EON[data_columns]

target=df_ml_EON['Answer_label']

# Train test split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data, target, test_size=0.2, random_state=1)

# Initialize a vectorizer object
vectorizer = CountVectorizer()

# Update the value of X_train and X_test
X_train = np.asarray(vectorizer.fit_transform(X_train['Comment_no_stopwords']).todense())
X_test = np.asarray(vectorizer.transform(X_test['Comment_no_stopwords']).todense())

# Create a clf classifier and train the model on the training set
clf = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0, max_depth=1, random_state=0).fit(X_train, y_train)

# Calculate predictions
y_pred = clf.predict(X_test)

# Confusion matrix
confusion_matrix = pd.crosstab(y_test, y_pred, rownames=['Actual class'], colnames=['Predicted clas'])
confusion_matrix


CPU times: total: 1min 3s
Wall time: 1min 7s


Predicted clas,0,1
Actual class,,
0,188,111
1,51,643


### Save the models locally

In [43]:
from joblib import Parallel, delayed 
import joblib 
  
  
# Save the model as a pickle in a file 
joblib.dump(vectorizer, 'vectorizer_EON_joblib.pkl') 
joblib.dump(clf, 'GradientBoostingClassifier_EON_joblib.pkl') 
    
# Load the model from the file 
#vectorizer = joblib.load('vectorizer_EON_joblib.pkl') 
#clf = joblib.load('GradientBoostingClassifier_EON_joblib.pkl') 
  
# Use the loaded model to make predictions 
#knn_from_joblib.predict(X_test) 

['GradientBoostingClassifier_EON_joblib.pkl']